### 1. CREAR BASE DE DATOS

* Crear con código SQL una base de datos llamada supermercado.
* Código Python con mysql connector que ejecute el SQL para borrar y generar la base de datos vacía.

In [1]:
import pandas as pd
import mysql.connector as con
import random
import datetime

Base de datos Supermercado

In [2]:
# Crear base de datos "supermercado"
conexion = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin"
)
cursor = conexion.cursor()
cursor.execute("DROP DATABASE IF EXISTS supermercado;")
cursor.execute("CREATE SCHEMA supermercado DEFAULT CHARACTER SET utf8;")
#cursor.close()
#connection.close()
#connection.close()

In [3]:
# Creo una sola tabla
# Conexion
conexion = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin",
    database= "supermercado"
)
cursor = conexion.cursor()

# crear tabla
query_tabla_tiendas = """
CREATE TABLE IF NOT EXISTS tiendas (
    id_tienda INT AUTO_INCREMENT PRIMARY KEY,
    nombre_tienda VARCHAR(30),
    direccion_tienda VARCHAR(50),
    ciudad VARCHAR (50)
)
"""
# Hacer los cambios en la base de datos
cursor.execute(query_tabla_tiendas)
conexion.commit()
print("Tabla creada exitosamente.")

# Cerrar la conexion
cursor.close()
conexion.close()

Tabla creada exitosamente.


In [4]:
# Creo varias tablas a la vez
conexion = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin",
    database= "supermercado"
)
cursor = conexion.cursor()

query_varias_tablas = ["""
    CREATE TABLE IF NOT EXISTS empleados (
    id_empleado INT AUTO_INCREMENT PRIMARY KEY,
    nombre_empleado VARCHAR(30),
    puesto VARCHAR(20),
    id_tienda INT,
    FOREIGN KEY (id_tienda) REFERENCES tiendas(id_tienda) ON DELETE CASCADE
)""",

    """
    CREATE TABLE IF NOT EXISTS categorias (
    id_categoria INT AUTO_INCREMENT PRIMARY KEY,
    nombre_categoria VARCHAR(40)
)""",

    """
    CREATE TABLE IF NOT EXISTS productos (
    id_producto INT AUTO_INCREMENT PRIMARY KEY,
    nombre_producto VARCHAR(40),
    precio DECIMAL (6, 2),
    stock INT,
    id_categoria INT,
    FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria) ON DELETE CASCADE
)""",

    """
    CREATE TABLE IF NOT EXISTS clientes (
    id_cliente INT AUTO_INCREMENT PRIMARY KEY,
    nombre VARCHAR(15),
    apellido VARCHAR (20),
    email VARCHAR (30),
    codigo_postal INT
)""",

    """
    CREATE TABLE IF NOT EXISTS ordenes (
    id_orden INT AUTO_INCREMENT PRIMARY KEY,
    id_cliente INT,
    FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente),
    id_empleado INT,
    FOREIGN KEY (id_empleado) REFERENCES empleados(id_empleado),
    fecha_orden DATE,
    metodo_pago ENUM('Tarjeta', 'Efectivo')
)""",

   """
    CREATE TABLE IF NOT EXISTS detalle_orden (
    id_detalle INT AUTO_INCREMENT PRIMARY KEY,
    id_orden INT NOT NULL,
    FOREIGN KEY (id_orden) REFERENCES ordenes(id_orden) ON DELETE CASCADE,
    id_producto INT NOT NULL,
    FOREIGN KEY (id_producto) REFERENCES productos(id_producto) ON DELETE CASCADE,
    cantidad INT (15) NOT NULL,
    descuento DECIMAL (3,2),
    precio_unitario DECIMAL(6,2) NOT NULL
)"""
]
# Para ejecutar cada consulta a kla base de datos y crear todas las tablas
for query in query_varias_tablas:
    cursor.execute(query)

conexion.commit()
print("Tablas creadas exitosamente.")

# Para copiar el "precio" de la tabla "productos" en la tabla "detalle_orden"
query_trigger = """
CREATE TRIGGER insertar_precio_unitario
BEFORE INSERT ON detalle_orden
FOR EACH ROW
BEGIN
    DECLARE precio_producto DECIMAL(6,2);

    -- Obtener el precio del producto desde la tabla productos
    SELECT precio INTO precio_producto 
    FROM productos 
    WHERE id_producto = NEW.id_producto;

    -- Asignar el precio al nuevo registro en detalle_orden
    SET NEW.precio_unitario = precio_producto;
END;
"""

cursor.execute(query_trigger)

conexion.commit()
print("Trigger creado exitosamente.")

# Para cerrar la conexión
cursor.close()
conexion.close()

Tablas creadas exitosamente.
Trigger creado exitosamente.


### 2. Generar datos demo desde Python

Tiendas

In [5]:
# Nombres de tiendas
filas = random.randint(11, 11)

nombre_tienda = [f'Tienda_0{numero}' if numero < 10 else f'Tienda_{numero}' for numero in range(filas)]
nombre_tienda

['Tienda_00',
 'Tienda_01',
 'Tienda_02',
 'Tienda_03',
 'Tienda_04',
 'Tienda_05',
 'Tienda_06',
 'Tienda_07',
 'Tienda_08',
 'Tienda_09',
 'Tienda_10']

In [6]:
# Direcciones 
direccion_tienda = [f'Calle_falsa_{100 + numero}' if numero < 10 else f'Calle_falsa_{100 + numero}' for numero in range(filas)]
direccion_tienda

['Calle_falsa_100',
 'Calle_falsa_101',
 'Calle_falsa_102',
 'Calle_falsa_103',
 'Calle_falsa_104',
 'Calle_falsa_105',
 'Calle_falsa_106',
 'Calle_falsa_107',
 'Calle_falsa_108',
 'Calle_falsa_109',
 'Calle_falsa_110']

In [7]:
# Lista de ciudades
ciudades = ["Madrid", "Barcelona", "Sevilla", "Valencia", "Bilbao", "Zaragoza", "Granada", "Alicante","Murcia","Albacete"]

# Número de ciudades aleatorias
num_ciudades = 10  

# Generar la lista de ciudades aleatorias
ciudades_aleatorias = [random.choice(ciudades) for _ in range(num_ciudades)]
print(ciudades_aleatorias)

['Granada', 'Sevilla', 'Barcelona', 'Bilbao', 'Albacete', 'Madrid', 'Albacete', 'Madrid', 'Barcelona', 'Barcelona']


In [8]:
# Insertamos los datos generados anteriormente
# Conectar a MySQL
conexion2 = con.connect(
    host="localhost",
    port= "3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion2.cursor()

# Insertamos los datos en la tabla
for i in range(10):
    cursor.execute("INSERT INTO tiendas (nombre_tienda, direccion_tienda, ciudad) VALUES (%s, %s, %s)",
                   (nombre_tienda[i], direccion_tienda[i], ciudades_aleatorias[i]))

conexion2.commit()
cursor.close()
conexion2.close()

Empleados

In [9]:
# Insetamos nombre empleados creados aleatoriamente desde un archivo CSV
# Conectar a MySQL
conexion3 = con.connect(
    host="localhost",
    port= "3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion3.cursor()

# Cargar el archivo CSV
df = pd.read_csv("nombres.csv")

# Insertar los datos en la tabla
for nombre in df["nombres"]:  # Asegúrate de que la columna se llama "nombre" en el CSV
    cursor.execute("INSERT INTO empleados (nombre_empleado) VALUES (%s)", (nombre,))

# Confirmar cambios
conexion3.commit()
cursor.close()
conexion3.close()

print("Datos importados correctamente a la base de datos.")

Datos importados correctamente a la base de datos.


In [10]:
# Lista de puestos
puestos = ["Director tienda", "Panadero", "Frutero", "Charcutero", "Pescatero", "Cajero", "Reponedor", "Repartidor"]

# Número de puestos aleatorias
num_puestos = 200  

# Generar la lista de puestos aleatorios
puestos_aleatorios = [random.choice(puestos) for _ in range(num_puestos)]
print(puestos_aleatorios)

['Frutero', 'Pescatero', 'Frutero', 'Director tienda', 'Pescatero', 'Frutero', 'Frutero', 'Reponedor', 'Reponedor', 'Reponedor', 'Director tienda', 'Panadero', 'Director tienda', 'Frutero', 'Cajero', 'Pescatero', 'Charcutero', 'Director tienda', 'Cajero', 'Pescatero', 'Panadero', 'Pescatero', 'Repartidor', 'Cajero', 'Cajero', 'Frutero', 'Reponedor', 'Frutero', 'Repartidor', 'Charcutero', 'Director tienda', 'Cajero', 'Pescatero', 'Pescatero', 'Frutero', 'Charcutero', 'Repartidor', 'Panadero', 'Repartidor', 'Cajero', 'Pescatero', 'Cajero', 'Charcutero', 'Pescatero', 'Pescatero', 'Panadero', 'Panadero', 'Director tienda', 'Repartidor', 'Director tienda', 'Panadero', 'Reponedor', 'Director tienda', 'Frutero', 'Director tienda', 'Frutero', 'Repartidor', 'Cajero', 'Reponedor', 'Repartidor', 'Panadero', 'Director tienda', 'Charcutero', 'Cajero', 'Panadero', 'Director tienda', 'Director tienda', 'Reponedor', 'Pescatero', 'Director tienda', 'Repartidor', 'Director tienda', 'Repartidor', 'Cajero

In [11]:
# Actualizamos tabla empleados
conexion4 = con.connect(
    host="localhost",
    port= "3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion4.cursor()

# Lista de IDs de tiendas disponibles en la tabla "tiendas"
cursor.execute("SELECT id_tienda FROM tiendas")
id_tienda_disponibles = [row[0] for row in cursor.fetchall()]

# Lista de empleados existentes
cursor.execute("SELECT id_empleado FROM empleados")
empleados = [row[0] for row in cursor.fetchall()]

# Actualizar cada empleado con un puesto y un id_tienda aleatorio
for id_empleado in empleados:
    puesto_aleatorio = random.choice(puestos_aleatorios)
    id_tienda_aleatorio = random.choice(id_tienda_disponibles)

    cursor.execute("UPDATE empleados SET puesto = %s, id_tienda = %s WHERE id_empleado = %s",
                   (puesto_aleatorio, id_tienda_aleatorio, id_empleado))

conexion4.commit()
cursor.close()
conexion4.close()

print("Los 200 empleados han sido actualizados correctamente con un puesto y una tienda aleatoria.")

Los 200 empleados han sido actualizados correctamente con un puesto y una tienda aleatoria.


Categorias

In [ ]:
# Insertar datos en la tabla categorias
conexion5 = con.connect(
    host="localhost",
    port= "3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion5.cursor()

categorias = ["Carnes", "Lacteos","Frutas", "Panaderia", "Bebidas", "Cereales", "Congelados", "Conservas", "Limpieza", "Belleza"]

# Número de categorias
num_categorias = 10  

# Generar la lista de categorias aleatorios
cat_aleatorias = [random.choice(categorias) for _ in range(num_categorias)]

# Insertar los datos en la tabla
for i in range(10):
    cursor.execute("INSERT INTO categorias (nombre_categoria) VALUES (%s)",
                   ([cat_aleatorias[i]]))

conexion5.commit()
cursor.close()
conexion5.close()

print("Las 10 categorias se han insertado correctamente")

Productos

In [ ]:
# Productos
productos = [
    "Leche deslactosada", "Yogur griego natural", "Queso mozzarella", "Mantequilla",
    "Pechuga de pollo", "Carne picada", "Jamón", "Salchichas",
    "Manzana roja", "Plátano", "Zanahoria","Lechuga",
    "Pan integral", "Croissants", "Galletas", "Pastel de chocolate",
    "Coca-Cola", "Zumo de naranja", "Café", "Agua mineral",
    "Arroz", "Lentejas", "Avena", "Fideos",
    "Helado de fresa", "Nuggets de pollo", "Espinacas congeladas", "Pizza congelada",
    "Atún", "Salsa de tomate", "Caballa", "Anchoas",
    "Detergente líquido", "Jabón de manos", "Papel higiénico","Desinfectante",
    "Champu", "Pasta de dientes", "Desodorante", "Crema hidratante"
    ]

# Número de categorias
num_productos = 40  

# Generar la lista de categorias aleatorios
cat_aleatorias = [random.choice(productos) for _ in range(num_productos)]


In [ ]:
# Precio productos